# Compare CPU and GPU image preprocessing in Spark OCR
* Load images from S3
* Preview images
* Recognize text

## Import OCR transformers and utils

In [0]:
from sparkocr.transformers import *
from sparkocr.enums import *
from sparkocr.databricks import display_images
from pyspark.ml import PipelineModel

## Define OCR transformers and pipeline
* Transforrm binary data to Image schema using [BinaryToImage](https://nlp.johnsnowlabs.com/docs/en/ocr_pipeline_components#binarytoimage). More details about Image Schema [here](https://nlp.johnsnowlabs.com/docs/en/ocr_structures#image-schema).
* Recognize text using [ImageToText](https://nlp.johnsnowlabs.com/docs/en/ocr_pipeline_components#imagetotext) transformer.

In [0]:
def pipeline():
    
    # Transforrm binary data to struct image format
    binary_to_image = BinaryToImage()
    binary_to_image.setInputCol("content")
    binary_to_image.setOutputCol("image")

    # Run OCR
    ocr = ImageToText()
    ocr.setInputCol("image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    
    pipeline = PipelineModel(stages=[
        binary_to_image,
        ocr
    ])
    
    return pipeline

In [0]:
def pipelineCPU():
    
    # Transforrm binary data to struct image format
    binary_to_image = BinaryToImage()
    binary_to_image.setInputCol("content")
    binary_to_image.setOutputCol("image")
    
    # Scale image
    scaler = ImageScaler()\
      .setInputCol("image")\
      .setOutputCol("scaled_image")\
      .setScaleFactor(2.0)
    
    # Binaraze image
    adaptive_thresholding = ImageAdaptiveBinarizer() \
      .setInputCol("scaled_image") \
      .setOutputCol("corrected_image") \
      .setMethod(ThresholdingMethod.OTSU)
    
    # Apply erosion
    erosion = ImageMorphologyOperation() \
      .setKernelShape(KernelShape.SQUARE) \
      .setKernelSize(1) \
      .setOperation("erosion") \
      .setInputCol("corrected_image") \
      .setOutputCol("eroded_image")

    # Run OCR
    ocr = ImageToText()
    ocr.setInputCol("eroded_image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    
    pipeline = PipelineModel(stages=[
        binary_to_image,
        scaler,
        adaptive_thresholding,
        erosion,
        ocr
    ])
    
    return pipeline

In [0]:
def pipelineGPU():
    
    # Transforrm binary data to struct image format
    binary_to_image = BinaryToImage()
    binary_to_image.setInputCol("content")
    binary_to_image.setOutputCol("image")
    
    # Image transformation on GPU
    transformer = GPUImageTransformer() \
      .addScalingTransform(2) \
      .addOtsuTransform() \
      .addErodeTransform(1, 1) \
      .setInputCol("image") \
      .setOutputCol("eroded_image") \

    # Run OCR
    ocr = ImageToText()
    ocr.setInputCol("eroded_image")
    ocr.setOutputCol("text")
    ocr.setConfidenceThreshold(65)
    
    pipeline = PipelineModel(stages=[
        binary_to_image,
        transformer,
        ocr
    ])
    
    return pipeline

## Download images from public S3 bucket to DBFS

In [0]:
%sh
OCR_DIR=/dbfs/tmp/ocr_1
if [ ! -d "$OCR_DIR" ]; then
    mkdir $OCR_DIR
    cd $OCR_DIR
    wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/ocr/datasets/news.2B.0.png.zip
    unzip news.2B.0.png.zip
fi

In [0]:
display(dbutils.fs.ls("dbfs:/tmp/ocr_1/0/"))

path,name,size
dbfs:/tmp/ocr_1/0/.DS_Store,.DS_Store,10244
dbfs:/tmp/ocr_1/0/8200_006.2B.png,8200_006.2B.png,101724
dbfs:/tmp/ocr_1/0/8200_006.2B.tif,8200_006.2B.tif,116158
dbfs:/tmp/ocr_1/0/8200_006.2B.txt,8200_006.2B.txt,3429
dbfs:/tmp/ocr_1/0/8200_006.2B.uzn,8200_006.2B.uzn,177
dbfs:/tmp/ocr_1/0/8200_014.2B.png,8200_014.2B.png,95597
dbfs:/tmp/ocr_1/0/8200_014.2B.tif,8200_014.2B.tif,116116
dbfs:/tmp/ocr_1/0/8200_014.2B.txt,8200_014.2B.txt,3309
dbfs:/tmp/ocr_1/0/8200_014.2B.uzn,8200_014.2B.uzn,87
dbfs:/tmp/ocr_1/0/8200_015.2B.png,8200_015.2B.png,64646


## Read images as binary files
from DBFS

In [0]:
images_path = "/tmp/ocr_1/0/*.png"
images_example_df = spark.read.format("binaryFile").load(images_path).cache()
display(images_example_df)

path,modificationTime,length,content
dbfs:/tmp/ocr_1/0/8230_014.2B.png,2020-10-05T08:32:01.000+0000,234628,iVBORw0KGgoAAAANSUhEUgAACKAAAAakCAAAAADsJq6YAAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOxd6ZrkKq4U95v3f2XuD4N2AXZmdbv6RMyczrLZxC4Fst06AQAAAAAAvAv/97cFAAAAAAAA8ICCAgAAAADA6wA= (truncated)
dbfs:/tmp/ocr_1/0/8230_022.2B.png,2020-10-05T08:32:02.000+0000,137300,iVBORw0KGgoAAAANSUhEUgAACKAAAAakAQAAAADhNszpAAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOy9bWwc2XnveZrNqDXXskoeXcScO3QXrQjyAjeINZqFxOzty1J2lFjGju5k98NFdpFg5CuBXmBlSbxjSPQu3VU= (truncated)
dbfs:/tmp/ocr_1/0/8250_007.2B.png,2020-10-05T08:31:50.000+0000,115888,iVBORw0KGgoAAAANSUhEUgAACKAAAAakCAAAAADsJq6YAAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOyd27bsGhBAOWP//y87DwmqKEIu3dJrzrH30klcigilkPjgAAAAANbiv28LAAAAAFCCggIAAADLgYICAAAAy4E= (truncated)
dbfs:/tmp/ocr_1/0/8202_024.2B.png,2020-10-05T08:31:55.000+0000,111865,iVBORw0KGgoAAAANSUhEUgAABrAAAAiYAQAAAAC3adqMAAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOy932/kyJ3gSTYLYi2gTZbPD5sNcxhl9OL8aAn9YBaGnWyjD+eXA/wf3Kkg4PrlcM5EHWDWdlqknEalB6tJ7gA= (truncated)
dbfs:/tmp/ocr_1/0/8200_006.2B.png,2020-10-05T08:31:54.000+0000,101724,iVBORw0KGgoAAAANSUhEUgAACKAAAAakCAAAAADsJq6YAAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOyd2batKAxFSY37/7+celAgCaGzxX3WHFX3uJVOpFkEROIAAAAAALAW/72dAAAAAAAACwQKAAAAAJYDAgUAAAA= (truncated)
dbfs:/tmp/ocr_1/0/8200_014.2B.png,2020-10-05T08:31:54.000+0000,95597,iVBORw0KGgoAAAANSUhEUgAABrAAAAiYCAAAAAC6ebj9AAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOyd6basKAxGodd9/1emfygQIEwOdUT3Xt33VCmTVlU+EyJaZwAAAJ7Pf389AAAAgBEQLAAAWAIECwAAlgDBAgA= (truncated)
dbfs:/tmp/ocr_1/0/8240_010.2B.png,2020-10-05T08:31:52.000+0000,74008,iVBORw0KGgoAAAANSUhEUgAACKAAAAakAQAAAADhNszpAAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOy9b3Ac15XYewfDZVMxzKbEV7tQjJ2GaRb8JbUBwS0SKs9Ow2+Zslyhnj+nXlKhVwiUDxAFhHnUKIanG4LLI74= (truncated)
dbfs:/tmp/ocr_1/0/8240_015.2B.png,2020-10-05T08:31:56.000+0000,68736,iVBORw0KGgoAAAANSUhEUgAABrAAAAiYAQAAAAC3adqMAAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOzd34/jSJ4g9mCxLln25onZmIfTYnWK6msY93jZ3gerAa1YjTLQMGDA/gMMXBbqYR6twgBeDVaTZI76imO4nNw= (truncated)
dbfs:/tmp/ocr_1/0/8200_015.2B.png,2020-10-05T08:31:49.000+0000,64646,iVBORw0KGgoAAAANSUhEUgAABrAAAAiYCAAAAAC6ebj9AAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOyd2bqtqA5GSX37/V85daFAEkJnt8T5j3NqL6dCCAgJnUocAAAAgPfz318rAAAAAIwAhwUAAGAJ4LAAAAAsARw= (truncated)
dbfs:/tmp/ocr_1/0/8250_010.2B.png,2020-10-05T08:32:00.000+0000,58333,iVBORw0KGgoAAAANSUhEUgAABrAAAAiYCAAAAAC6ebj9AAAACXBIWXMAAB7CAAAewgFu0HU+AAAgAElEQVR4nOydbberKAxG4a75/3+Z+aBAAgHxtXLce83c0yoC1TaPCQF9cAAAAO/n3687AAAAMAKCBQAAU4BgAQDAFCBYAAA= (truncated)


## Read data from s3 directly using credentials

In [0]:
# ACCESS_KEY = ""
# SECRET_KEY = ""
# sc._jsc.hadoopConfiguration().set("fs.s3n.awsAccessKeyId", ACCESS_KEY)
# sc._jsc.hadoopConfiguration().set("fs.s3n.awsSecretAccessKey", SECRET_KEY)
# imagesPath = "s3a://dev.johnsnowlabs.com/ocr/datasets/news.2B/0/*.tif"
# imagesExampleDf = spark.read.format("binaryFile").load(imagesPath).cache()
# display(imagesExampleDf)

## Display count of images

In [0]:
images_example_df.count()

Out[9]: 18

## Preview images using _display_images_ function

In [0]:
display_images(BinaryToImage().transform(images_example_df), limit=3)

Origin: dbfs:/tmp/ocr_1/0/8230_014.2B.png 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1 
 
 
 
 
 
 
 
 Origin: dbfs:/tmp/ocr_1/0/8230_022.2B.png 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1 
 
 
 
 
 
 
 
 Origin: dbfs:/tmp/ocr_1/0/8250_007.2B.png 
 Resolution: 199 dpi 
 Width: 2208 px 
 Height: 1700 px 
 Mode: Greyscale 
 Number of channels: 1

## Run OCR pipeline without preprocessing

In [0]:
result = pipeline().transform(images_example_df.repartition(8)).cache()
result.count()

Out[11]: 18

In [0]:
display(result.select("text", "confidence"))

text confidence N. Korea: ‘We don’t want war’

Aging leader denies
nuclear weapons
are being built

The Associated Press

BEIJING — North Korean Presi-
dent Kim Il-sung said Saturday that
he has no plans to develop nuclear
weapons and that he wants peace,
according to a CNN correspondent on
a rare visit to the reclusive Stalinist
state.

CNN’s Beijing bureau chief, Mike
Chinoy, quoted Kim as saying, “We
have done a lot of construction in our
country, and we don’t want to destroy
it.

“Those who want war are out of
their minds.”

Chinoy and correspondents from
The Washington Times and the
Japanese television network NHK
were invited to North Korea for

celebrations of Kim's 82nd birthday,
which was Friday.

In a telephone call from the capital,
Pyongyang, Chinoy said the commu-
nist leader seemed to be trying to
reduce tensions that have built up in
recent weeks since the North Koreans
again refused to allow a full interna-
tional inspection of nuclear facilities.

Kim criticized as “out of place” a
threat made by one of his own
officials in March to turn the South
Korean capital, Seoul, into a “sea of
fire.”

“We don’t want war,” he said.

South Korea, Japan and the West
fear that North Korea, suspected of
international terrorism in the past, is
developing or already has developed
nuclear weapons.

During a 24-hour meeting and
lunch with the journalists and other
foreign visitors, Kim said repeatedly
that North Korea has neither the
capability nor the desire to build up a

nuclear arsenal, according to Chinoy.

“The world is now calling on our
country to show the nuclear weapons
we don't have,” Chinoy quoted Kim
as saying.

“What's the use to have them? We
cannot use nuclear weapons against
our Korean nationals” in South
Korea.

He said his country lacks the
missile systems to direct nuclear
weapons against more distant targets.

“Our country is a small country.
We have no place to carry out
testing,” he said.

However, Chinoy said Kim did not
say anything about whether experts
from the International Atomic Energy
Agency would be allowed to return
for even the limited inspections they
have made before.

The U.N. Security Council for-
mally asked North Korea two weeks
ago to allow full inspections, but
North Korea rejected the request.
 95.24774627685547 ‘Supermax’ home for feds’

FLORENCE — Beneath 34-foot-high
guard towers, workmen are putting the
finishing touches on ‘‘Supermax,”’ also
known as “the Aleatraz of the Rockies.”

This is the part of the federal prison
complex that will house the nation’s most
notorious, most dangerous and most en-
dangered male prisoners.

It is intended to augment or possibly
supplant the “maximum-custody” prison
in Marion, Ili., which in turn replaced the
federal penal island of Alcatraz, in the
middle of San Francisco Bay, in 1963.

Because of the legendary heritage of Al-
catraz and, to a lesser degree, Marion, Su-
permax in Florence is already rich in folk-
lore, even though it won’t open for several
more months.

Warden Bill Story, 54, is anxious to dis-
pel what he calls the public’s misunder-
standings about ‘U.S. Penitentiary /Ad-
ministrative Maximum,” Supermax’s
formal name.

Contrary to popular belief, Story said,
most of the 556 prisoners at Supermax
will not be kept in their single-cot cells 23
hours a day. Nor will all its prisoners, of-
ten referred to as ‘‘the worst of the worst,”
be required to eat in their cells. And they
won’t all be prohibited from attending re-
ligious services or other group activities.

Individual cells in Supermax’s “control
unit” will house 64 inmates 23 hours a day,
as will a separate 72-cell segregation unit,
which Story calls “a jail within a jail.” In-
mates held in this rigid system of confine-
ment will be permitted one hour a day out-
side their cells to exercise.

The other inmates will have access to
group activities and be permitted out of
their cells for anywhere from three to 12
hours a day.

Supermax also will have four VIP cells
should it 

## CPU image preprocessing

In [0]:
resultCPU = pipelineCPU().transform(images_example_df.repartition(8)).cache()
resultCPU.count()

Out[13]: 18

In [0]:
display(resultCPU.select("text", "confidence"))

text confidence ‘Supermax’ home for feds’

FLORENCE — Beneath 34-foot-high
guard towers, workmen are putting the
finishing touches on ‘‘Supermax,” also
known as “the Aleatraz of the Rockies.”

This is the part of the federal prison
complex that wii] house the nation’s most
notorious, most dangerous and most en-
dangered male prisoners.

It is intended to augment or possibly
supplant the “maximum-custody” prison
in Marion, Ili., which in turn replaced the
federal penal island of Alcatraz, in the
middle of San Francisco Bay, in 1963.

Because of the legendary heritage of Al-
catraz and, to a lesser degree, Marion, Su-
permax in Florence is already rich in folk-
lore, even though it won’t open for severai
more months.

Warden Bill Story, 54, is anxious to dis-
pel what he calls the public’s misunder-
standings about “U.S. Penitentiary /Ad-
ministrative Maximum,” Supermax’s
formal name.

Contrary to popular belief, Story said,
most of the 556 prisoners at Supermax
will not be kept in their single-cot cells 23
hours a day. Nor will all its prisoners, of-
ten referred to as ‘‘the worst of the worst,”
be required to eat in their cells. And they
won't all be prohibited from attending re-
ligious services or other group activities.

Individual cells in Supermax’s “control
unit” will house 64 inmates 23 hours a day,
as will a separate 72-cell segregation unit,
which Story calls “a jail within a jail.” In-
mates held in this rigid system of confine-
ment will be permitted one hour a day out-
side their cells to exercise.

The other inmates will have access to
group activities and be permitted out of
their cells for anywhere from three to 12
hours a day.

Supermax also will have four VIP cells
should it receive an inmate with extraor-

about the plight of American Indians staged a peaceful vigil

Unlike Marion, where the worst prison-
ers are held in basement cells, all cells at
Supermax have windows. In Marion, cells

Still, “you can never say this is escape-
proof, because that could come back to
haunt you,” said Story.

 

The Denver Post / Helen H. Dav
PRISON PROTEST: About 125 protesters concerned _ outside the federal prison complex in this 1992 photo. One «

the major concerns was the treatment of inmates.

been manipulators all of their lives an
they can work people,” said Story, who
actively recruiting guards with extensiy

 78.92200724283855 Occult-crime
expert shines
light on Satanism

The Toledo Blade

TIFFIN, Ohio — Dale Griffis,
who has emerged as one of the
nation’s leading experts on occult
crime, returned home from the
most historic case of his career
wishing there were no need for his
services.

“I personally would like to be
put out of business,” said the
former Tiffin police captain, whose
work, experts say, has changed the
complexion of occult-related activ-
ity in both the court and class-
room.

“Yd like to have this whole
bizarre topic just go away.”

Griffis, who has a degree in
criminal justice, appeared on
ABC-TV’s 20/20 in 1985. He now
is sought as lecturer and expert
witness in occult trials across the
country.

Last month, prosecutors in
Jonesboro, Ark., called Griffis as
an expert witness in a triple-mur-
der trial that police and prosecu-
tors believe will redefine the
courtroom treatment of occultism.
Griffis spent seven hours on the
witness stand testifying that the
1993 slaying of three 8-year-old
boys — whose bodies were found
naked, hogtied and battered —
were rituals of human sacrifice.

Deputy prosecutor John Fogle-
man called on Griffis to testify that
Damien Wayne Echols, 18, and
Curtis Jason Baldwin, 16, killed
the second-graders while enacting
occult ideology. After leaving
Jonesboro under armed guard,
Griffis returned home to watch the
verdicts on television, Echols was
sentenced to death by injection.
Baldwin was sentenced to life in
prison without parole.

It is frightening, Griffis said, to
realize that those who kill under
the guise of Satanic ideology do so
for greater power and glory in he

## GPU image preprocessing

In [0]:
resultGPU = pipelineGPU().transform(images_example_df.repartition(8)).cache()
resultGPU.count()

Out[15]: 18

In [0]:
display(resultGPU.select("text", "confidence"))

text confidence N. Korea: ‘We don’t want war’

Aging leader denies
nuclear weapons
are being built

The Associated Press

BEDJING — North Korean Presi-
dent Kim Il-sung said Saturday that
he has no plans to develop nuclear
weapons and that he wants peace,
according to a CNN correspondent on
a rare visit to the reclusive Stalinist
state.

CNN’s Beijing bureau chief, Mike
Chinoy, quoted Kim as saying, “We
have done a lot of construction in our
country, and we don’t want to destroy
it.

“Those who want war are out of
their minds.”

Chinoy and correspondents from
The Washington Times and the
Japanese television network NHK
were invited to North Korea for

celebrations of Kim's 82nd birthday,
which was Friday.

In a telephone call from the capital,
Pyongyang, Chinoy said the commu-
nist leader seemed to be trying to
reduce tensions that have built up in
recent weeks since the North Koreans
again refused to allow a full interna-
tional inspection of nuclear facilities.

Kim criticized as “out of place” a
threat made by one of his own
officials in March to turn the South
Korean capital, Seoul, into a “sea of
fire.”

“We don’t want war,” he said.

South Korea, Japan and the West
fear that North Korea, suspected of
international terrorism in the past, is
developing or already has developed
nuclear weapons.

During a 24-hour meeting and
lunch with the journalists and other
foreign visitors, Kim said repeatedly
that North Korea has neither the
capability nor the desire to build up a

nuclear arsenal, according to Chinoy.

“The world is now calling on our
country to show the nuclear weapons
we don't have,” Chinoy quoted Kim
as saying.

“What’s the use to have them? We
cannot use nuclear weapons against
our Korean nationals” in South
Korea.

He said his country lacks the
missile systems to direct nuclear
weapons against more distant targets.

“Our country is a small country.
We have no place to carry out
testing,” he said.

However, Chinoy said Kim did not
say anything about whether experts
from the International Atomic Energy
Agency would be allowed to return
for even the limited inspections they
have made before.

The U.N. Security Council for-
mally asked North Korea two weeks
ago to allow full inspections, but
North Korea rejected the request.
 94.95111999511718 

FLORENCE — Beneath 34-foot-high
guard towers, workmen are putting the
finishing touches on “Supermax,” also
known as “the Aleatraz of the Rockies.”

This is the part of the federal prison
complex that wil] house the nation’s most
notorious, most dangerous and most en-
dangered male prisoners.

It is intended to augment or possibly
supplant the “maximum-custody” prison
in Marion, Ili., which in turn replaced the
federal penal island of Alcatraz, in the
middie of San Francisco Bay, in 1963.

Because of the legendary heritage of Al-
catraz and, to a lesser degree, Marion, Su-
permax in Florence is already rich in folk-
lore, even though it won’t open for several
more months.

Warden Bill Story, 54, is anxious to dis-
pel what he calls the public’s misunder-
standings about “U.S. Penitentiary /Ad-
ministrative Maximum,” Supermax’s
formal name.

Contrary to popular belief, Story said,
most of the 556 prisoners at Supermax
will not be kept in their single-cot cells 23
hours a day. Nor will all its prisoners, of-
ten referred to as “the worst of the worst,”
be required to eat in their cells. And they
won't all be prohibited from attending re-
ligious services or other group activities.

Individual cells in Supermax’s “control
unit” will house 64 inmates 23 hours a day,
as will a separate 72-cell segregation unit,
which Story calls “a jail within a jail.” In-
mates held in this rigid system of confine-
ment will be permitted one hour a day out-
side their cells to exercise.

The other inmates will have access to
group activities and be permitted out of
their cells for anywhere from three to 12
hours a day.

Supermax also will have four VIP cells
should it receive an inmate with extra